# 1. Configurar ambiente

In [2]:
import pandas as pd
import numpy as np
import re
import os
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
!pip install jellyfish
import jellyfish
!pip install transformers
from transformers import BertForSequenceClassification
from transformers import BertTokenizer
import spacy
!python -m spacy download es_core_news_md
nlp = spacy.load('es_core_news_md')
from bs4 import BeautifulSoup
import requests
import torch
import random

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 14.7 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 46.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 48.7 MB/s eta 0:00:00
2023-05-27 20:59:53.254236: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 MB 16.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_md')


#2. Tratamiento de datos

In [3]:
#Función para encontrar la raiz de las palabras
def raiz(palabra):
  radio=0
  palabra_encontrada=palabra
  for word in lista_verbos:
    confianza = jellyfish.jaro_winkler(palabra, word)
    if (confianza>=0.92 and confianza>=radio):
      radio=confianza
      palabra_encontrada=word
  return palabra_encontrada

def tratamiento_texto(texto):
  trans = str.maketrans('áéíóú','aeiou')
  texto = texto.lower()
  texto = texto.translate(trans)
  texto = re.sub(r"[^\w\s]", '', texto)
  texto = " ".join(texto.split())
  return texto

#Función para normalizar la palabra
def normalizar(texto):
  doc = nlp(texto)
  tokens=[]
  if len(doc)<=3:
    for t in doc:
      if t.pos_=='VERB':
        tokens.append(raiz(t.lemma_))
      else:
        tokens.append(t.lemma_)
  else:
    for t in doc:
      if (t.pos_ in ('VERB','PROPN','PRON','NOUN','AUX','SCONJ','DET','ADJ','ADV') or any(t.dep_.startswith(elemento) for elemento in ['ROOT'])):
        if t.pos_=='VERB':
          tokens.append(raiz(t.lemma_))
        else:
          tokens.append(t.lemma_)
  tokens = list(dict.fromkeys(tokens))
  tokens = tokens[:10]
  tokens = ' '.join(tokens)
  return tratamiento_texto(tokens)

# 3. Cargar bases de conocimiento

In [4]:
#Importando verbos en español
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36'}
trans = str.maketrans('áéíóú','aeiou')
lista_verbos=[]
url = ['https://www.ejemplos.co/verbos-mas-usados-en-espanol/',
       'https://www.ejemplos.co/tipos-de-verbos/',
       'https://www.ejemplos.co/verbos-predicativos/',
       'https://www.ejemplos.co/verbos-personales/',
       'https://www.ejemplos.co/verbos-irregulares/',
       'https://www.ejemplos.co/verbos/',
       'https://www.ejemplos.co/100-ejemplos-de-verbos-regulares/',
       'https://www.ejemplos.co/verbos-del-decir/',
       'https://www.ejemplos.co/verbos-con-a/',
       'https://www.ejemplos.co/verbos-con-b/',
       'https://www.ejemplos.co/verbos-con-c/',
       'https://www.ejemplos.co/verbos-con-d/',
       'https://www.ejemplos.co/verbos-con-e/',
       'https://www.ejemplos.co/verbos-con-f/',
       'https://www.ejemplos.co/verbos-con-g/',
       'https://www.ejemplos.co/verbos-con-h/',
       'https://www.ejemplos.co/verbos-con-i/',
       'https://www.ejemplos.co/verbos-con-j/',
       'https://www.ejemplos.co/verbos-con-k/',
       'https://www.ejemplos.co/verbos-con-l/',
       'https://www.ejemplos.co/verbos-con-ll/',
       'https://www.ejemplos.co/verbos-con-m/',
       'https://www.ejemplos.co/verbos-con-n/',
       'https://www.ejemplos.co/verbos-con-o/',
       'https://www.ejemplos.co/verbos-con-p/',
       'https://www.ejemplos.co/verbos-con-q/',
       'https://www.ejemplos.co/verbos-con-r/',
       'https://www.ejemplos.co/verbos-con-s/',
       'https://www.ejemplos.co/verbos-con-t/',
       'https://www.ejemplos.co/verbos-con-u/',
       'https://www.ejemplos.co/verbos-con-v/',
       'https://www.ejemplos.co/verbos-con-w/',
       'https://www.ejemplos.co/verbos-con-x/',
       'https://www.ejemplos.co/verbos-con-y/',
       'https://www.ejemplos.co/verbos-con-z/']

for i in range(len(url)):
  try:
    respuesta = requests.get(url[i], headers=headers)
    respuesta = respuesta.content.decode('utf-8')
    bases = pd.read_html(respuesta)
    for i, df in enumerate(bases):
      for idx,row in bases[i].iterrows():     
          _ = [lista_verbos.append(re.sub(r"\((.*?)\)", '', x.lower()).strip().translate(trans)) for x in row[0].split('/')]
          _ = [lista_verbos.append(re.sub(r"\((.*?)\)", '', x.lower()).strip().translate(trans)) for x in row[1].split('/')]
          _ = [lista_verbos.append(re.sub(r"\((.*?)\)", '', x.lower()).strip().translate(trans)) for x in row[2].split('/')]
  except Exception:
    continue
lista_verbos = [elemento for elemento in lista_verbos if len(elemento) != 2]
lista_verbos=list(set(lista_verbos))

#Importando bases de dialogo fluído
txt_folder_path = '/content'
lista_documentos=[x for x in os.listdir(txt_folder_path) if x.endswith(".txt")]
lista_dialogos, lista_dialogos_respuesta, lista_tipo_dialogo = [],[],[]
for idx in range(len(lista_documentos)):
  f=open(txt_folder_path+'/'+lista_documentos[idx], 'r', encoding='utf-8', errors='ignore')
  flag,posicion = True,0
  for line in f.read().split('\n'):
    if flag:
      line = tratamiento_texto(line)
      lista_dialogos.append(line)
      lista_tipo_dialogo.append(lista_documentos[idx].replace('.txt', ''))
    else:
      lista_dialogos_respuesta.append(line)
      posicion+=1
    flag=not flag

#Creando Dataframe de diálogos
datos = {'dialogo':lista_dialogos,'respuesta':lista_dialogos_respuesta,'tipo':lista_tipo_dialogo,'interseccion':0,'similarity':0,'jaro_winkler':0,'probabilidad':0}
df_dialogo = pd.DataFrame(datos)
df_dialogo = df_dialogo.drop_duplicates(keep='first')
df_dialogo.reset_index(drop=True, inplace=True)

# 4. Buscar respuesta del Chatbot

In [38]:
#Función para verificar si el usuário inició un diálogo
def dialogo(user_response):
  df = df_dialogo.copy()
  vectorizer = TfidfVectorizer()
  dialogos_numero = vectorizer.fit_transform(df_dialogo['dialogo'])
  respuesta_numero = vectorizer.transform([user_response])
  for idx,row in df.iterrows():
    df.at[idx,'interseccion'] = len(set(user_response.split()) & set(row['dialogo'].split()))/len(user_response.split())
    df.at[idx,'similarity'] = cosine_similarity(dialogos_numero[idx], respuesta_numero)[0][0]
    df.at[idx,'jaro_winkler'] = jellyfish.jaro_winkler(user_response,row['dialogo'])
    df.at[idx,'probabilidad'] = max(df.at[idx,'interseccion'],df.at[idx,'similarity'],df.at[idx,'jaro_winkler'])
  df.sort_values(by=['probabilidad','jaro_winkler'], inplace=True, ascending=False)
  return df.head(3)

#Cargar el modelo entrenado
ruta_modelo = '/content/modelo'
Modelo_TF = BertForSequenceClassification.from_pretrained(ruta_modelo)
tokenizer_TF = BertTokenizer.from_pretrained(ruta_modelo)

def clasificacion_modelo(pregunta):
  frase = normalizar(pregunta)
  tokens = tokenizer_TF.encode_plus(
      frase,
      add_special_tokens=True,
      max_length=128,
      padding='max_length',
      truncation=True,
      return_tensors='pt'
  )
  input_ids = tokens['input_ids']
  attention_mask = tokens['attention_mask']

  with torch.no_grad():
      outputs = Modelo_TF(input_ids, attention_mask)

  etiquetas_predichas = torch.argmax(outputs.logits, dim=1)
  etiquetas_decodificadas = etiquetas_predichas.tolist()

  diccionario = {3: 'Continuacion', 10: 'Nombre', 2: 'Contacto', 13: 'Saludos', 14: 'Sentimiento', 9: 'Identidad', 15: 'Usuario', 6: 'ElProfeAlejo', 1: 'Aprendizaje', 0: 'Agradecimiento', 5: 'Edad', 4: 'Despedida', 11: 'Origen', 12: 'Otros', 7: 'Error', 8: 'Funcion'}
  llave_buscada = etiquetas_decodificadas[0]
  clase_encontrada = diccionario[llave_buscada]

  #Buscar respuesta más parecida en la clase encontrada
  df = df_dialogo[df_dialogo['tipo'] == clase_encontrada]
  df.reset_index(inplace=True)
  vectorizer = TfidfVectorizer()
  dialogos_num = vectorizer.fit_transform(df['dialogo'])
  pregunta_num = vectorizer.transform([tratamiento_texto(pregunta)])
  similarity_scores = cosine_similarity(dialogos_num, pregunta_num)
  indice_pregunta_proxima = similarity_scores.argmax()
  return clase_encontrada, df['respuesta'][indice_pregunta_proxima]

# 5. Ejecutar Chatbot

In [70]:
pregunta='que es machine learning?'
user_response = tratamiento_texto(pregunta)
respuesta = dialogo(user_response)
respuesta

,dialogo,respuesta,tipo,interseccion,similarity,jaro_winkler,probabilidad
971,que es la ia,La Inteligencia Artificial (IA) es un campo de...,Otros,0.5,0.535875,0.887681,0.887681
308,que es el amor,El amor es un sentimiento profundo de afecto y...,Sentimiento,0.5,0.523251,0.867989,0.867989
963,que es api,API es el acrónimo de Application Programming ...,Otros,0.5,0.559535,0.858261,0.858261


In [71]:
clase = clasificacion_modelo(pregunta)
print(clase[0])
print(clase[1])

Otros
Python es un lenguaje de programación interpretado cuya filosofía hace hincapié en la legibilidad de su código. Fue creado por Guido van Rossum, y liberado al público en 1991. Python tiene una sintaxis clara y concisa, lo que la convierte en un lenguaje ideal para la programación en general. Está diseñado para ser fácilmente legible, lo que lo hace ideal para la prototipación y el desarrollo de proyectos a gran escala.
